In [ ]:
# Python Script: Online Price Comparator with Selenium 🛒

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# Sample product to search
product = "kit podcaster"

# Websites to compare (using simplified demo selectors)
websites = {
    "Amazon": {
        "url": "https://www.amazon.com",
        "search_box_selector": "input[name='field-keywords']",
        "result_selector": "span.a-price-whole"
    },
    "MercadoLibre": {
        "url": "https://www.mercadolibre.com",
        "search_box_selector": "input[name='as_word']",
        "result_selector": "span.andes-money-amount__fraction"
    }
}

# Set up Chrome driver with options
options = Options()
options.add_argument('--headless')  # Run browser in background
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920x1080')
driver = webdriver.Chrome(options=options)

results = []

# Iterate over each website
for site, data in websites.items():
    try:
        # Navigate to site
        driver.get(data["url"])
        time.sleep(2)

        # Find the search box and enter product
        search_box = driver.find_element(By.CSS_SELECTOR, data["search_box_selector"])
        search_box.clear()
        search_box.send_keys(product)
        search_box.send_keys(Keys.RETURN)
        time.sleep(3)

        # Find the first price element
        price_element = driver.find_element(By.CSS_SELECTOR, data["result_selector"])
        price_text = price_element.text.replace(',', '').replace('.', '')
        price = float(price_text.strip())

        # Append to results
        results.append({
            "Website": site,
            "Price": price,
            "Link": driver.current_url
        })

    except Exception as e:
        print(f"Error with {site}: {e}")
        continue

# Close the driver
driver.quit()

# Create DataFrame
df = pd.DataFrame(results)

# Find the best price
best = df.loc[df["Price"].idxmin()]

# Show all results
print("All Prices:\n", df)

# Show best deal
print("\nBest Deal:")
print(best)

# Visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.bar(df["Website"], df["Price"], color=["#1f77b4", "#ff7f0e"])
plt.title("Product Price Comparison")
plt.ylabel("Price (USD)")
plt.xlabel("Website")
plt.grid(True)
plt.tight_layout()
plt.show()
